In [3]:
import numpy as np
import pandas as pd
from numba import njit, objmode, prange,deferred_type,cuda,jit,vectorize
from numba.typed import Dict
from numba.core import types
import time
import string
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt

from BPnumba.DFFA import createDFFA,DFFA
from BPnumba.EDFFA import createEDFFA,EDFFA
from BPnumba.DABC import DABC, createDABC
from BPnumba.EDABC import createEDABC ,EDABC
from BPnumba.AGH import createAG,NAG,FlipMutation,GASearch
from BPnumba.BoxN import create_ItemBin,ItemBin,InstaceBoxes
from BPnumba.BPPH import create_Bin
from BPnumba.Individual import Ind, createR_intidivual,create_intidivual
from BPnumba.Poblation import CreateHeuristicPob,CreateRotHPob,EvalPob
from BPmodule.BPGen import createPA,GetInstance,createInstance,createPA
from BPnumba.BPPH import create_Bin

from collections import OrderedDict
from numba.experimental import jitclass

In [10]:
@vectorize
def flip(x, y,pm,rot):
    if x > pm:
        return y
    else:
        if y == rot-1:
            return 0
        else: 
            return y+1

In [13]:
N = 50
pr=0.05
a = np.random.random(size=N)
b= np.random.randint(6,size=N)
print(a)
print(b)
flip(a,b,pr,6)

[0.16207851 0.77948275 0.6468888  0.56016312 0.250563   0.99328
 0.58858219 0.87483056 0.80140388 0.76316056 0.49400012 0.79508214
 0.31324797 0.30198822 0.95901865 0.44970795 0.66052773 0.07171209
 0.85257936 0.76700852 0.95586765 0.11861663 0.40196916 0.36594549
 0.52985575 0.52442671 0.46754826 0.50588427 0.98582557 0.28730356
 0.29437554 0.12372438 0.48109602 0.90712343 0.22041397 0.71872817
 0.46713991 0.11966419 0.83448931 0.57019945 0.56016847 0.695987
 0.01541194 0.62185005 0.46416676 0.24636051 0.21847737 0.34202147
 0.63184653 0.48518453]
[1 1 5 3 0 5 4 3 2 2 0 1 5 5 4 3 4 1 5 3 5 5 2 5 2 3 2 4 5 0 4 2 0 0 1 5 1
 0 2 2 0 1 4 1 5 5 3 1 5 5]


array([1, 1, 5, 3, 0, 5, 4, 3, 2, 2, 0, 1, 5, 5, 4, 3, 4, 1, 5, 3, 5, 5,
       2, 5, 2, 3, 2, 4, 5, 0, 4, 2, 0, 0, 1, 5, 1, 0, 2, 2, 0, 1, 5, 1,
       5, 5, 3, 1, 5, 5], dtype=int64)

In [87]:
@cuda.jit
def add_array(a, b, c,pm):
    i = cuda.threadIdx.x + cuda.blockDim.x * cuda.blockIdx.x
    if i < a.size:
        if a[i]>pm:
            c[i] = a[i] - b[i]
        else:
            c[i] = 1
@njit
def FlipMutation(gen, pm:float,rotType:int):
    for i in np.arange(len(gen)):
        r = random.random()
        if r<=pm:
            if rotType == 6:
                aux = np.random.randint(rotType)
                if gen[i] == aux:
                    gen[i]+=1
                    if gen[i] >= rotType-1:
                        gen[i]=0
            else:
                if(gen[i]==0):
                    aux=1
                else:
                    aux=0
            gen[i] = aux
N = 50
pr=0.05
a = np.random.random(size=N)
b = a
print(a,b)
dev_c = cuda.device_array_like(a)

add_array[4, 8](a, b, dev_c,pr)
c = dev_c.copy_to_host()
print(c)

[0.31139646 0.86448227 0.57615888 0.5297334  0.99172375 0.27043266
 0.92948283 0.85397985 0.11069715 0.92877429 0.2596887  0.71735791
 0.78442787 0.02935487 0.29324495 0.08690028 0.72285279 0.03570446
 0.11984323 0.50865444 0.10230579 0.95414199 0.49332303 0.71570601
 0.43922725 0.64614872 0.39736241 0.24952235 0.58859367 0.64713195
 0.43267527 0.31347628 0.58655255 0.15287351 0.68309687 0.67558027
 0.88736261 0.16001072 0.3601936  0.39780867 0.20245679 0.12618371
 0.34308089 0.22229879 0.49062042 0.30534101 0.91332603 0.64819481
 0.40653863 0.84440678] [0.31139646 0.86448227 0.57615888 0.5297334  0.99172375 0.27043266
 0.92948283 0.85397985 0.11069715 0.92877429 0.2596887  0.71735791
 0.78442787 0.02935487 0.29324495 0.08690028 0.72285279 0.03570446
 0.11984323 0.50865444 0.10230579 0.95414199 0.49332303 0.71570601
 0.43922725 0.64614872 0.39736241 0.24952235 0.58859367 0.64713195
 0.43267527 0.31347628 0.58655255 0.15287351 0.68309687 0.67558027
 0.88736261 0.16001072 0.3601936  0.39

c:\Users\NicolasG\anaconda3\lib\site-packages\numba\cuda\compiler.py:726: NumbaPerformanceWarning: Grid size (4) < 2 * SM count (76) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\NicolasG\anaconda3\lib\site-packages\numba\cuda\cudadrv\devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [88]:
@cuda.jit
def FlipMutation(a, b, c,pm,rot):
    i = cuda.grid(1)
    if i < a.size:
        if a[i]>pm:
            c[i] = b[i]
        else:
            if c[i]==rot:
                c[i] = 0
            else:
                c[i] = b[i]+1
@jit
def mn(b):
    N = 50     
    a = np.random.random(size=N)
   
    print(b)
    dev_a = cuda.to_device(a)
    dev_b = cuda.to_device(b)

    threads_per_block = 256
    blocks_per_grid = (N + (threads_per_block - 1)) // threads_per_block
    dev_c = cuda.device_array_like(b)
    FlipMutation[blocks_per_grid, threads_per_block](dev_a, dev_b, dev_c,0.05,6)

    c = dev_c.copy_to_host()
    print(c)

In [96]:
mn(np.random.randint(7,size=50))

[3 4 5 0 5 1 0 6 1 6 0 0 4 2 0 0 4 6 5 0 1 5 5 2 6 0 1 5 5 1 5 0 5 6 0 5 5
 0 4 0 2 5 5 1 1 2 2 2 1 1]
[3 4 5 0 5 1 0 6 1 6 0 0 4 2 0 0 4 6 5 0 1 5 5 3 6 0 1 5 5 1 5 0 5 6 0 5 5
 0 4 0 2 5 6 1 1 2 2 2 1 1]


In [ ]:
rn = np.random.randint(50,size=50,dtype=np.int64)
rn

In [ ]:
np.random.random(size=50)

In [ ]:
BR1 = GetInstance("BR1")
contenedor = BR1[0]
DataSet = BR1[1][0]

In [ ]:
from BPnumba.BPPH import DBLF
from BPnumba.Individual import CalcFi


@njit
def mn(cont, data,rot):
    UBOXES = InstaceBoxes(data)
    hpob = CreateRotHPob(pop =20, ITEMS = UBOXES.copy(), rotType=rot)
    EvalPob(hpob,UBOXES,cont,rot)
    ind = hpob[0]
    print(ind.fi)

In [ ]:
mn(contenedor, DataSet,6)

In [ ]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=" ",
    fmt='%g'
)

numTest = 20
num = 5
agL = zerovec(num, 2*numTest)
dabcL = zerovec(num, 2*numTest)
ffaL = zerovec(num, 2*numTest)

pr_selec = 0.85
pr_cross = 0.75
maxItr=1000
pop = 50

In [ ]:

@njit(parallel=True)
def BenchAG(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    for i in prange(20):
        alg:NAG = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=0,mutType=heuristic)
        alg.Train(maxItr,  InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), NumbaList(boxes[i]), bin)
        lst[i] =alg.BestInd.fi

def TestAG(algorithms, problems, tp):
    nm:string=""
    if tp==1:
        nm="c1HAG"
        print("Empieza",nm)
    elif tp == 2:
        nm="c2HAG"
        print("Empieza",nm)
    else:
        nm="HAG"
        print("Empieza",nm)
    for problem in problems:
        for algoritm in algorithms:
            
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            BenchAG(datos.copy(),contenedor, maxItr, agL,tp)
            if problem!=1 and algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/AG/"+nm,agL)

In [ ]:
@njit(parallel=True)
def BenchABC(boxes:List[List[List[int]]],bin:List[int],maxItr: int, lst: List[List[float]], heuristic:int=0):
    n=len(boxes[0])
    for i in prange(20):
        if(heuristic==0):
            alg:DABC = createDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi

        else:
            alg:EDABC = createEDABC()
            alg.Train(numItr=maxItr,ColonyWorker=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0),datos= NumbaList(boxes[i]),contenedor= bin)
            lst[i] =alg.BestInd.fi
def TestEDABC(algorithms, problems,tp:int):
    nm:string=""
    if tp != 0:
        nm="c2EDABC"
        print("Inicia c2EDABC")
    else:
        nm="DABC"
        print("Inicia DABC")
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            BenchABC(datos.copy(),contenedor, maxItr, agL,heuristic =tp)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/ABC/"+nm,agL)
           

In [ ]:
@njit(parallel=True)
def BenchFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0
    ):
  
    for i in prange(20):
        alg:DFFA = createDFFA(mutType=0)
        alg.Train(Maxitr=maxItr,fireflyPob=InstancePob(CreateHeuristicPob(50,NumbaList(boxes[i]),bin),NumbaList(boxes[i]), bin, 0), datos= NumbaList(boxes[i]),contenedor= bin)
        lst[i] =alg.BestInd.fi

@njit(parallel=True)
def BenchEFFA(
    boxes:List[List[List[int]]],
    bin:List[int],
    maxItr: int, 
    lst: List[List[float]],
    tp:int=0,
    updateIndex:int=5,
    ):
   for i in prange(20): 
        alg:EDFFA = createEDFFA(mutType=0)
        alg.Train(updateIndex,maxItr, InstancePob(NumbaList( CreateHeuristicPob(pop,boxes[i],True)),boxes[i],bin),boxes[i],bin)
        lst[i] =alg.BestInd.fi

In [ ]:
def TestFFA(
    algorithms:list[int], 
    problems:list[int],
    tp:int, 
    evolutive:bool,
    updateIndex:int
    ):
    nm:string = ""
    if evolutive:
        if tp==1:
            nm="c1EDFFA"
            print("Inicia",nm)
        elif  tp==2:
            nm="c2EDFFA"
            print("Inicia", nm)
        else:
            nm="EDFFA"
            print("Inicia",nm)
    else:
        if tp==1:
            nm="c1DFFA"
            print("Inicia",nm)
        elif tp==2:
            nm="c2DFFA"
            print("Inicia",nm)
        elif tp==3:
            nm="invDFFA"
            print("Inicia",nm)
        else:
            nm="DFFA"
            print("Inicia",nm)
   
    for problem in problems:
        for algoritm in algorithms:
            agL = zerovec(1, numTest)
            contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
            start_time = time.time()
            if not evolutive:
                BenchFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp)
            else:
                BenchEFFA(boxes=datos,bin=contenedor, maxItr=maxItr,lst= agL,tp=tp,updateIndex=updateIndex)
            seconds = np.abs( start_time - time.time())
            hours = int(seconds/3600.0) 
            minutes = int(seconds/60) % 60
            sec = int(seconds % 60)
            print("Termina P", problem, "A",algoritm," | ",hours,"h :",minutes,"m :",sec,"s")
            if algoritm!=1: createText("P"+str(problem)+"A"+str(algoritm)+"/FFA/"+nm,agL)

In [ ]:
def Compile():
    UPDATE_INDEX:int=10
    ALGORITHM=[1]
    PROBLEM=[1]
    for _ in np.arange(4):
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM,
            tp=_,
            evolutive=True,
            updateIndex=UPDATE_INDEX,
        )
        if _ == 3: continue
        TestFFA(
            algorithms=ALGORITHM,
            problems=PROBLEM, 
            tp=_,
            evolutive=False,
            updateIndex=UPDATE_INDEX,
        )
    TestEDABC(ALGORITHM,PROBLEM,0)
    TestEDABC(ALGORITHM,PROBLEM,1)

In [ ]:
Compile()

In [ ]:
TestEDABC([2],[3],0)

In [ ]:

TestEDABC([2],[3],1)


In [ ]:
UPDATE_INDEX:int=10


TestFFA(
        algorithms=[4],
        problems=[2],
        tp=0,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

TestFFA(
        algorithms=[4],
        problems=[2],
        tp=1,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )
TestFFA(
        algorithms=[4],
        problems=[2],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
    )

In [ ]:

TestFFA(
        algorithms=[2],
        problems=[3],
        tp=2,
        evolutive=True,
        updateIndex=UPDATE_INDEX
)

TestEDABC([2],[4],0)
TestEDABC([2],[4],1)

TestFFA(
    algorithms=[2],
    problems=[4],
    tp=0,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=1,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
    algorithms=[2],
    problems=[4],
    tp=2,
    evolutive=True,
    updateIndex=UPDATE_INDEX
)
TestFFA(
        algorithms=[2],
        problems=[4],
        tp=0,
        evolutive=False,
        updateIndex=UPDATE_INDEX
)